In [1]:
from datasets import load_dataset
import pandas as pd
mbpp = load_dataset("mbpp")

No config specified, defaulting to: mbpp/full


Generating train split:   0%|          | 0/374 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/90 [00:00<?, ? examples/s]

Generating prompt split:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset mbpp downloaded and prepared to /home/chloe/.cache/huggingface/datasets/mbpp/full/1.0.2/4458a31cd4305553c8e88e3f0bfb94fc74fe1a9faeeb8c32ed166939735eaeff. Subsequent calls will reuse this data.


  0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
mbpp

DatasetDict({
    train: Dataset({
        features: ['task_id', 'text', 'code', 'test_list', 'test_setup_code', 'challenge_test_list'],
        num_rows: 374
    })
    test: Dataset({
        features: ['task_id', 'text', 'code', 'test_list', 'test_setup_code', 'challenge_test_list'],
        num_rows: 500
    })
    validation: Dataset({
        features: ['task_id', 'text', 'code', 'test_list', 'test_setup_code', 'challenge_test_list'],
        num_rows: 90
    })
    prompt: Dataset({
        features: ['task_id', 'text', 'code', 'test_list', 'test_setup_code', 'challenge_test_list'],
        num_rows: 10
    })
})

In [4]:
mbpp_train_df = pd.DataFrame(mbpp["train"])
mbpp_train_df.head()

,task_id,text,code,test_list,test_setup_code,challenge_test_list
0,601,Write a function to find the longest chain whi...,"class Pair(object): \r\n\tdef __init__(self, a...","[assert max_chain_length([Pair(5, 24), Pair(15...",,[]
1,602,Write a python function to find the first repe...,"def first_repeated_char(str1):\r\n for index,...","[assert first_repeated_char(""abcabc"") == ""a"", ...",,[]
2,603,Write a function to get a lucid number smaller...,def get_ludic(n):\r\n\tludics = []\r\n\tfor i ...,"[assert get_ludic(10) == [1, 2, 3, 5, 7], asse...",,[]
3,604,Write a function to reverse words in a given s...,def reverse_words(s):\r\n return ' '.jo...,"[assert reverse_words(""python program"")==(""pro...",,[]
4,605,Write a function to check if the given integer...,def prime_num(num):\r\n if num >=1:\r\n for...,"[assert prime_num(13)==True, assert prime_num(...",,[]


In [18]:
mbpp_prompt_df = pd.DataFrame(mbpp["prompt"])
mbpp_test_df = pd.DataFrame(mbpp["test"])
print(mbpp_prompt_df.iloc[3]['text'])
print(mbpp_test_df.iloc[3]['test_list'])

Write a function to find the largest integers from a given list of numbers using heap queue algorithm.
['assert find_Volume(10,8,6) == 240', 'assert find_Volume(3,2,2) == 6', 'assert find_Volume(1,2,1) == 1']


In [19]:
max_list_length = mbpp_test_df['test_list'].apply(len).max()
max_list_length

3

In [20]:
# format the test df such that it uses the same format as the coqllm dataset
mbpp_test_df[['test_case_1', 'test_case_2', 'test_case_3']] = mbpp_test_df['test_list'].apply(
    lambda x: pd.Series(x[:3] + [None] * max(0, 3 - len(x)))
)
mbpp_test_coq = mbpp_test_df[['text', 'test_case_1', 'test_case_2', 'test_case_3']]
mbpp_test_coq = mbpp_test_coq.rename(columns={'text': 'specification'})
# remove "assert" statements from the test cases
test_case_columns = ['test_case_1', 'test_case_2', 'test_case_3']
def remove_asserts(text):
    if text is None:
        return None
    return text.replace('assert ', '')
def fix_equals(text):
    if text is None:
        return None
    return text.replace('==', '=')

mbpp_test_coq[test_case_columns] = mbpp_test_coq[test_case_columns].applymap(remove_asserts)
mbpp_test_coq[test_case_columns] = mbpp_test_coq[test_case_columns].applymap(fix_equals)

mbpp_test_coq.head()


,specification,test_case_1,test_case_2,test_case_3
0,Write a python function to remove first and la...,"remove_Occ(""hello"",""l"") == ""heo""","remove_Occ(""abcda"",""a"") == ""bcd""","remove_Occ(""PHP"",""P"") == ""H"""
1,Write a function to sort a given matrix in asc...,"sort_matrix([[1, 2, 3], [2, 4, 5], [1, 1, 1]])...","sort_matrix([[1, 2, 3], [-2, 4, -5], [1, -1, 1...","sort_matrix([[5,8,9],[6,4,3],[2,1,4]])==[[2, 1..."
2,Write a function to count the most common word...,"count_common(['red','green','black','pink','bl...","count_common(['one', 'two', 'three', 'four', '...","count_common(['Facebook', 'Apple', 'Amazon', '..."
3,Write a python function to find the volume of ...,"find_Volume(10,8,6) == 240","find_Volume(3,2,2) == 6","find_Volume(1,2,1) == 1"
4,Write a function to split a string at lowercas...,"split_lowerstring(""AbCd"")==['bC','d']","split_lowerstring(""Python"")==['y', 't', 'h', '...","split_lowerstring(""Programming"")==['r', 'o', '..."
